In [1]:
import os
import shutil
import sys
import time

import numpy as np
import copy

from dolfinx import fem, mesh, io, nls
from dolfinx import plot as dofplot
from ufl import dx, ds, grad, inner, exp, sin, diff, div
import ufl
from mpi4py import MPI
from petsc4py import PETSc

from ipywidgets import IntProgress
from IPython.display import display
import matplotlib.pyplot as plt
import pyvista as pv

In [2]:
class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)

In [3]:
dot =Infix(inner)

In [4]:
def draw_plot(space, points, func):
    c_topology, c_cell_types, c_geometry = dofplot.create_vtk_mesh(space)
    check = np.vstack((np.arange(len(func)), func)).T[points][:, 1]
    c_data = np.column_stack((c_geometry[:, 0:2], check))
    x_data = c_data[:, 0]
    y_data = c_data[:, 1]
    z_data = c_data[:, 2]
    plt.tripcolor(x_data, y_data, z_data, shading='gouraud')

In [5]:
doamain_mesh = mesh.create_rectangle(
    points=((0, 0), (2, 3)),
    n=(2, 3),
    cell_type=mesh.CellType.triangle,
    ghost_mode=mesh.GhostMode.none,
    comm=MPI.COMM_WORLD,
    )
u_elem = ufl.VectorElement("Lagrange", doamain_mesh.ufl_cell(), 2)
p_elem = ufl.FiniteElement("Lagrange", doamain_mesh.ufl_cell(), 1)
u_space, p_space = fem.FunctionSpace(doamain_mesh,u_elem),fem.FunctionSpace(doamain_mesh,p_elem)

In [6]:
bc_static_X = np.zeros(doamain_mesh.geometry.dim, dtype=PETSc.ScalarType)
bc_static_marker = mesh.locate_entities_boundary(
    doamain_mesh,
    dim=1,
    marker=lambda x: np.isclose(x[0], 0)|np.isclose(x[0], 1)
    |np.isclose(x[1], 0)
    )
bc_static = fem.dirichletbc(
    V=u_space,
    value=bc_static_X,
    dofs=fem.locate_dofs_topological(
        V=u_space, entity_dim=1, entities=bc_static_marker
        ),
    )


def bc_dynamic_X_expression(x):
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1])))


bc_dynamic_X = fem.Function(u_space)
bc_dynamic_X.interpolate(bc_dynamic_X_expression)
bc_dynamic_marker = mesh.locate_entities_boundary(
    doamain_mesh,
    dim=1,
    marker=lambda x: np.isclose(x[1], 1),
    )
bc_dynamic = fem.dirichletbc(
    bc_dynamic_X, fem.locate_dofs_topological(u_space, 1, bc_dynamic_marker)
    )

bcs = [bc_static, bc_dynamic]

In [7]:
u, p = ufl.TrialFunction(u_space), ufl.TrialFunction(p_space)
v, q = ufl.TestFunction(u_space), ufl.TestFunction(p_space)

f_ufl = ufl.Constant(doamain_mesh, (PETSc.ScalarType(0), PETSc.ScalarType(0)))
f_fem = fem.Constant(doamain_mesh, (PETSc.ScalarType(0), PETSc.ScalarType(0)))

g_ufl = ufl.Constant(doamain_mesh, PETSc.ScalarType(0))
g_fem = fem.Constant(doamain_mesh, PETSc.ScalarType(0))

a_block = fem.form(
    [
        [(grad(u)|dot|grad(v)) * dx, (p|dot|div(v)) * dx],
        [(div(u)|dot|q) * dx, None]
        ]
    )
a_p11 = fem.form((p|dot|q) * dx)
a_p = [[a_block[0][0], None], [None, a_p11]]
L_block = fem.form([(f_fem|dot|v) * dx, (g_fem|dot|q) * dx])

In [10]:
a_p11

### Nested matrix solver

In [9]:
assebled = fem.petsc.assemble_matrix_nest(a_block,bcs=bcs)
assebled.assemble()